# Finding Causality in Big Data: An Introduction

### Shirin Mojarad, Ph.D.
### Data Scientist, McGraw-Hill Education

@shirinmojarad

Shirin.mojarad@gmail.com

https://github.com/Lewkow/LAK_2017_Workshop

# Agenda 

* Introduction to causality
* How to establish causality
* Randomized controlled trials
* Causal inference
* Your Toolkit For Finding Causality

# Your Toolkit For Finding Causality

# Causal Inference in 5 Steps

<ol type="1">
<li>Identify covariates from observational data</li>
<li>Choose a matching metric </li>
<li>Execute a matching algorithm</li>
<li>Examine covariate balance</li>
<li>Estimate treatment effects </li>
</ol>

# Data

We want to study the effect of a training program on individuals’ earnings.

* Data are from the National Supported Work Demonstration (NSWD) and the Current Population Survey (Dehejia and Wahba survey 2002)
* Treatment is if a person received training 
* Independent variables are age, education, marrital status, earnings in 1974 and 1975, employement status in 1974 and 1975
* Outcome is real earnings in 1978 (RE78)
* We need to find matches for the 185 treated observations and then compare outcomes

## Import required libraries and data

In [8]:
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import binom, hypergeom, gaussian_kde
from scipy.stats import ttest_ind
import math
from scipy.stats import binom, hypergeom, gaussian_kde
import matplotlib.patches as mpatches

In [9]:
df = sqlContext.sql("SELECT * FROM matching_earnings_1").toPandas()
print(df.shape)
df.head()

In [10]:
print 'Number of treated units: %d' %df.groupby('TREAT').size()[1]
print 'Number of control units: %d' %df.groupby('TREAT').size()[0]

Note from the output that not all of the control observations might be used as matches for the 185 treated observations.

## Convert numeric columns to integer type

In [13]:
# several numeric columns are shown with type object instead of integer
df.dtypes

In [14]:
# function to convert dataframe column types to float
def convertFloat(data):
  for attribute in data.columns:
    data[attribute] = data[attribute].astype(float) 
  return data

In [15]:
# apply the above function
df = convertFloat(df)

## Define treatment, outcome and covariates

In [17]:
# treatment
Tr = df.TREAT

# outcome
Y = df.EARNINGS
Y0 = df.EARNINGS[df.TREAT==0]
Y1 = df.EARNINGS[df.TREAT==1]

# covariates
X = df[['AGE', 'EDUC', 'MARR','NODEGREE','RE74','RE75','U74','U75']]

# 1 Identify covariates from observational data
## 1.1 Difference-in-means: outcome variable

* Effectiveness of treatment (effect on RE78 expression)
* Question: does treatment affect RE78 expression (outcome)?

In [19]:
df.groupby('TREAT')['EARNINGS'].mean().reset_index()

In [20]:
# Mean difference 
print 'Differene in means: $'+ str(int(Y1.mean() - Y0.mean())) 

It seems that individuals who did not receive training earn $15k on average more than those who received the training.

In [22]:
# t-statistic & p-value for difference in outcome of two groups
def hypothesisTest(data, attribute, group):
    x = data[attribute][df[group]==1]
    y = data[attribute][df[group]==0]
    # t-statistic & p-value for difference in outcome of two groups
    t = ttest_ind(x, y)[0]
    p = ttest_ind(x, y)[1]
    # Confidence intervals
    cm = sms.CompareMeans(sms.DescrStatsW(x), sms.DescrStatsW(y))
    CI = str(round(cm.tconfint_diff(usevar='unequal')[0]))+' - '+str(round(cm.tconfint_diff(usevar='unequal')[1]))
    # Cohen's d
    pooledvar = math.sqrt((pow(x.std(),2) + (pow(y.std(),2)))/2)
    d = (x.mean()-y.mean()) / pooledvar
    # create dataframe
    tablelist = []
    tablerow = [attribute,x.mean()-y.mean(),t,p,CI,d]
    tablelist.append(tablerow)
    out = pd.DataFrame(tablelist)
    out.columns = ['Attribute','Mean Difference','t-value','p-value','95% Confidence Intervals',"Cohen's d"]

    return out

In [23]:
hypothesisTest(df, 'EARNINGS', 'TREAT')

The difference-in-means is significant and the effect size is large.

# 1.3 Difference-in-means: pre-treatment covariates
### We’ll work with the following covariates:

* Age
* Education
* Race (Black/White)
* Marital status
* Whether or not the subject has a college degree
* Whether or not the subject was unemployed in 1974 and 1975
* Earnings in 1974 and 1975

Let’s get the difference-in-means for each of these covariates, by the treatment status:

In [26]:
def plotDisribution(data, attribute, groups):
  f, ax = plt.subplots(figsize=(10, 6))
  pre = pd.DataFrame({'groups':data[groups], 'propensity':data[attribute]})    

  density0 = gaussian_kde(pre.propensity[pre.groups==0])
  density1 = gaussian_kde(pre.propensity[pre.groups==1])
  xs = np.linspace(0,data[attribute].max(),1000)
  ax.plot(xs,density0(xs),color='blue')
  ax.fill_between(xs,density1(xs),color='gray')
  ax.set_title('Before Matching')
  ax.set_xlabel(attribute)
  ax.set_ylabel('Density')
  
  gray_patch = mpatches.Patch(color='gray', label='Treated')
  blue_patch = mpatches.Patch(color='blue', label='Not Treated')
  plt.legend(handles=[gray_patch,blue_patch], loc='center left',bbox_to_anchor=(1, 0.5),fontsize=8)
  
  return(f)

In [27]:
f=plotDisribution(df,'AGE', 'TREAT'); display(f)

In [28]:
f=plotDisribution(df,'EDUC', 'TREAT'); display(f)

In [29]:
# Exercise: plot the distribution for the rest of covariate for treatment and control groups
f=plotDisribution(df,'NODEGREE', 'TREAT'); display(f)

In [30]:
f=plotDisribution(df,'RE74', 'TREAT'); display(f)

In [31]:
f=plotDisribution(df,'RE75', 'TREAT'); display(f)

In [32]:
# difference of means for each attribute, by treatment group
df.groupby('TREAT').mean().reset_index()

* What do you see? Take a moment to reflect on what these differences suggest for the relationship of interest (that between treated and not treated).

We can carry out t-tests to evaluate whether these means are statistically distinguishable:

In [35]:
b = pd.DataFrame()
for att in X:
    a = []
    a = hypothesisTest(df, att, 'TREAT')
    b = pd.concat([b,a],0)
b

## 2 Propensity score matching

Match treated and untreated observations on the estimated probability of being treated (propensity score)

P(X) = Pr (Tr=1|X)

We can estimate the propensity scores using logistic regression.

In [37]:
####### Using GLM
import statsmodels.genmod.generalized_linear_model as sm
glm_binom = sm.GLM(Tr, X, family=sm.families.Binomial())
res = glm_binom.fit()
print res.summary()
propensityScoreGLM = res.fittedvalues
df = pd.concat([df[['TREAT']],X,df[['EARNINGS']],propensityScoreGLM],1)
df.columns = np.concatenate(( ['TREAT'],X.columns,['EARNINGS' ,'Propensity Score']), axis=0)
print '\n'+ 'Propensity Scores added: '
df.head()

## 3 Execute a matching algorithm

There are several variants of matching: 
* one-to-one matching
* one-to-many matching 
* with or without a caliper, and with or without replacement

Variants of the methods are examined in the following paper:

</br> Austin, P. C. (2014), A comparison of 12 algorithms for matching on the propensity score. Statist. Med., 33: 1057–1069. doi: 10.1002/sim.6004

In [39]:
# one-to-one matching
N = len(Tr)
N1 = Tr[Tr == 1].index; N2 = Tr[Tr == 0].index
g1, g2 = propensityScoreGLM[Tr == 1], propensityScoreGLM[Tr == 0]

## Check if treatment groups got flipped - the smaller should correspond to N1/g1
if len(N1) > len(N2):
   N1, N2, g1, g2 = N2, N1, g2, g1

## match treatment and control group
matches = {}
caliper = False
replace = False

for m in N1:
    dist = abs(g1[m] - g2)
    if (dist.min() <= caliper) or not caliper:
            matches[m] = dist.argmin()  
            if not replace:
                g2 = g2.drop(matches[m])

print 'Sample matches: '
{k: matches[k] for k in matches.keys()[:5]}

In [40]:
# Matched data:
tr = matches.keys()
ctrl = matches.values()
temp = pd.concat([df.ix[tr],df.ix[ctrl]])
df_matched = temp.groupby(temp.index).first()
df_matched.groupby('TREAT').size().reset_index()

## 4 Examine covariate balance
### 4.1 Visual inspection

Plot density of propensity scores and covariates for each group before and after matching

In [42]:
from scipy.stats import binom, hypergeom, gaussian_kde
import matplotlib.patches as mpatches
def plotMatching(data, attribute, groups, matches):
  f, ax = plt.subplots(1, 2, figsize=(10, 6))
  pre = pd.DataFrame({'groups':data[groups], 'propensity':data[attribute]})    

  tr = matches.keys()
  ctrl = matches.values()
  temp = pd.concat([pre.ix[tr], pre.ix[ctrl]])
  post = temp.groupby(temp.index).first()


  density0 = gaussian_kde(pre.propensity[pre.groups==0])
  density1 = gaussian_kde(pre.propensity[pre.groups==1])
  xs = np.linspace(0,data[attribute].max(),1000)
  ax[0].plot(xs,density0(xs),color='blue')
  ax[0].fill_between(xs,density1(xs),color='gray')
  ax[0].set_title('Before Matching')
  ax[0].set_xlabel(attribute)
  ax[0].set_ylabel('Density')

  density0_post = gaussian_kde(post.propensity[post.groups==0])
  density1_post = gaussian_kde(post.propensity[post.groups==1])
  xs = np.linspace(0,data[attribute].max(),1000)
  ax[1].plot(xs,density0_post(xs),color='blue')
  ax[1].fill_between(xs,density1_post(xs),color='gray')
  ax[1].set_title('After Matching')
  ax[1].set_xlabel(attribute)
  ax[1].set_ylabel('Density')
  
  gray_patch = mpatches.Patch(color='gray', label='Treatment')
  blue_patch = mpatches.Patch(color='blue', label='Control')
  plt.legend(handles=[gray_patch,blue_patch], loc='center left',bbox_to_anchor=(1, 0.5),fontsize=8)
  
  return(f)

In [43]:
f = plotMatching(df, 'Propensity Score', 'TREAT', matches);display(f)

In [44]:
f = plotMatching(df, 'AGE', 'TREAT', matches);display(f)

In [45]:
f = plotMatching(df, 'EDUC', 'TREAT', matches);display(f)

In [46]:
f = plotMatching(df, 'MARR', 'TREAT', matches);display(f)

In [47]:
f = plotMatching(df, 'NODEGREE', 'TREAT', matches);display(f)

In [48]:
f = plotMatching(df, 'RE74', 'TREAT', matches);display(f)

In [49]:
f = plotMatching(df, 'RE75', 'TREAT', matches);display(f)

In [50]:
f = plotMatching(df, 'U74', 'TREAT', matches);display(f)

In [51]:
f = plotMatching(df, 'U75', 'TREAT', matches);display(f)

### 4.2 Difference-in-means

In [53]:
def balanceCheck(data, attribute, group):
    means = data[[group, attribute]].groupby(group).mean().reset_index()
    dist = abs(means.diff()).ix[1]
    std = data[[group, attribute]].groupby(group).std().reset_index()
    n = data[group].value_counts()
    se = std.apply(lambda(s): np.sqrt(s[0]**2/n[0] + s[1]**2/n[1]))
    tablelist = []
    tablerow = [attribute,dist[1],se[1]]
    tablelist.append(tablerow)
    out = pd.DataFrame(tablelist)
    out.columns = ['Attribute','difference of means by group','Standard error for covariates by group']
    return out

In [54]:
b = pd.DataFrame()
for att in X:
    a = []
    a = balanceCheck(df, att, 'TREAT')
    b = pd.concat([b,a],0)
b

In [55]:
b = pd.DataFrame()
for att in X:
    a = []
    a = hypothesisTest(df_matched, att, 'TREAT')
    b = pd.concat([b,a],0)
b

Difference in means for covariates is not significant anymore and the effect sizes are negligible.

## 5 Estimate treatment effects

Estimate average treatment effect  on the treated

### 5.1 Computes ATT using difference in means

In [58]:
# Matched outcome
Y0_matched = df_matched.EARNINGS[df.TREAT == 0]
Y1_matched = df_matched.EARNINGS[df.TREAT == 1]

print 'Differene in means: $'+ str(int(Y1_matched.mean() - Y0_matched.mean())) 